## Projet Statistique


Ce projet a été fait par Enzo RABAHI, Léon DEBEVER, Lorenzo COUZINET et Quentin DOMON. Il consiste a utiliser les statistiques pour résoudre des problèmes de cryptographie. Ce projet va suivre la même structure que le document pdf qui nous a été fourni par Mme.CAUWET.

On va utiliser plusieurs bibliothèque python tel que numpy, et chisquare pour effectuer le test du chi-carré


In [ ]:
import numpy as np
from scipy.stats import chisquare

: 

On va commencer par importer les fichiers textes qui nous importe, c'est-à-dire les fichiers .txt

In [ ]:
freq = np.loadtxt("frequence.txt")

: 

## 1.1 Chiffrement par substitution et code de César

On utilise donc le fichier str.txt qui nous est fourni et que l'on va essayer de décrypter de manière itérative sachant qu'il y a 26 possibilités. On peut se permettre de faire une boucle qui va modifier le message en faisant une indentation de 1 à chaque fois.

In [10]:
f = open('str.txt', 'r')
contenu = f.read()
print(contenu)
f.close()

'gurpnrfnepvcurevfbarbsgurrneyvrfgxabjanaqfvzcyrfgpvcurefvgvfnglcrbsfhofgvghgvbapvcurevajuvpurnpuyrggrevagurcynvagrkgvffuvsgrqnpregnvaahzorebscynprfqbjagurnycunorg'



On peut voir ici à quoi ressemble le fichier texte en question. Il est normal qu'il nous parle pas puisqu'il est crypté

In [38]:
def cesar(txt, k):
    contenu = ""
    
    # 1. On ouvre le fichier
    f = open(txt, 'r')
    # 2. On lit TOUT le fichier d'un coup (pour avoir une chaine de caractères)
    texte_entier = f.read()
    f.close() # On ferme le fichier

    # 3. On parcourt chaque lettre
    for i in texte_entier:
        
        # LOGIQUE DE DÉCRYPTAGE :
        # On prend le code ASCII et on retire le décalage
        code_lettre = ord(i) - k 

        # GESTION DE LA BOUCLE (si on sort de l'alphabet)
        # Le code ASCII de 'a' est 97. Si en reculant on descend en dessous de 97, il faut rajouter 26pour repartir de la fin de l'alphabet ('z').
        if code_lettre < 97:
            code_lettre = code_lettre + 26
            
        contenu += chr(code_lettre)

    return contenu

In [ ]:
for k in range (0,26,1):
    print(cesar('str.txt',k)'/n')

: 

In [25]:
print(cesar('str.txt',13))

4thecaesarcipherisoneoftheearliestknownandsimplestciphersitisatypeofsubstitutioncipherinwhicheachletterintheplaintextisshiftedacertainnumberofplacesdownthealphabet4


Après vérification, on peut donc voir que le code césar à déchiffrer était d'un décalage de 14 vers la droite. 

On a parcouru l'entièreté des possibilités sachant qu'il y avait très peu (26). Cette méthode, certes efficace, ne peut être véritablement pris en compte de par la petitesse du nombre de cas.

On sait maintenant ce que ce code signifie, on va pouvoir passer au 1.3, c'est-à-dire au test du chi-carré

## 1.3 Test du CHI-carré


In [ ]:
import numpy as np

# --- 1. CHARGEMENT DES DONNÉES ---

# Chargement des fréquences théoriques de l'anglais (données en pourcentages)
cite_start=[]# [cite: 30, 31, 32]
try:
    freq_th = np.loadtxt('frequence.txt')
except OSError:
    # Valeurs de secours si le fichier n'est pas présent (basées sur le fichier fourni)
    freq_th = np.array([8.2, 1.5, 2.8, 4.3, 12.7, 2.2, 2.0, 6.1, 7.0, 0.15, 
                        0.77, 4.0, 2.4, 6.7, 7.5, 1.9, 0.095, 6.0, 6.3, 9.1, 
                        2.8, 0.98, 2.4, 0.15, 2.0, 0.074])

# Chargement du message chiffré
[cite_start]# [cite: 23, 147]
try:
    with open('str.txt', 'r') as f:
        message_chiffre = f.read().strip()
except OSError:
    # Message contenu dans le fichier str.txt fourni
    message_chiffre = 'gurpnrfnepvcurevfbarbsgurrneyvrfgxabjanaqfvzcyrfgpvcurefvgvfnglcrbsfhofgvghgvbapvcurevajuvpurnpuyrggrevagurcynvagrkgvffuvsgrqnpregnvaahzorebscynprfqbjagurnycunorg'

# --- 2. DÉFINITION DES FONCTIONS (Partie 2.1 du sujet) ---

def dechiffre_texte(texte, k):
    """
    Déchiffre un texte chiffré par décalage de César.
    Entrées : texte (str), k (int - la clé)
    Sortie : texte clair (str)
    [cite_start][cite: 89]
    """
    resultat = []
    # On parcourt chaque caractère du message
    for car in texte:
        # [cite_start]On ne traite que les lettres minuscules (a-z) comme indiqué dans le sujet [cite: 25]
        if 'a' <= car <= 'z':
            # [cite_start]1. On convertit le caractère en code ASCII (a=97) [cite: 84]
            code_ascii = ord(car)
            # 2. On remet l'index à 0-25 (donc on soustrait 97)
            index = code_ascii - 97
            # 3. On applique le décalage INVERSE (gauche) pour déchiffrer : index - k
            # Le modulo 26 (%) gère le retour à la fin de l'alphabet si on descend sous 0
            nouvel_index = (index - k) % 26
            # [cite_start]4. On reconvertit en caractère [cite: 86]
            nouveau_car = chr(nouvel_index + 97)
            resultat.append(nouveau_car)
        else:
            # Si ce n'est pas une lettre, on le laisse tel quel (ex: chiffres, symboles si présents)
            resultat.append(car)
            
    return "".join(resultat)

def compte_frequence(texte):
    """
    Calcule la fréquence d'apparition de chaque lettre (a-z) dans le texte.
    Renvoie un tableau numpy de taille 26 avec les pourcentages.
    [cite_start][cite: 91]
    """
    compteur = np.zeros(26) # Tableau de 26 zéros pour stocker les comptes
    total_lettres = 0
    
    for car in texte:
        if 'a' <= car <= 'z':
            index = ord(car) - 97
            compteur[index] += 1
            total_lettres += 1
            
    # Si le texte est vide, on évite la division par zéro
    if total_lettres == 0:
        return compteur

    # On convertit en pourcentage pour correspondre au format de frequence.txt
    [cite_start]# [cite: 92]
    freq_obs = (compteur / total_lettres) * 100
    return freq_obs

# --- 3. DÉCRYPTAGE AUTOMATIQUE (Partie 1.3 du sujet) ---

print("--- Analyse automatique par test du Chi-2 ---")

scores_chi2 = [] # Pour stocker le score de chaque clé

# [cite_start]On teste toutes les clés possibles de 0 à 25 [cite: 74]
for k in range(26):
    # [cite_start]1. Déchiffrer le texte avec la clé k [cite: 74]
    texte_candidat = dechiffre_texte(message_chiffre, k)
    
    # [cite_start]2. Calculer les fréquences du texte candidat [cite: 75]
    freq_obs = compte_frequence(texte_candidat)
    
    # [cite_start]3. Calcul du Chi-carré [cite: 76, 101]
    # Formule : somme((Observé - Théorique)² / Théorique)
    # Note : On ajoute une petite valeur epsilon (1e-9) pour éviter la division par zéro si une fréquence théorique était nulle (ce qui n'est pas le cas ici mais c'est une sécurité).
    chi2 = np.sum(((freq_obs - freq_th) ** 2) / (freq_th + 1e-9))
    
    scores_chi2.append(chi2)
    
    # Affichage optionnel pour voir le processus
    # print(f"Clé {k:2d} : Score Chi2 = {chi2:.2f} -> Début: {texte_candidat[:20]}...")

# --- 4. RÉSULTATS ---

# Convertir la liste en tableau numpy pour faciliter la recherche
scores_chi2 = np.array(scores_chi2)

# La meilleure clé est celle qui a le score Chi2 le plus BAS (le plus proche de 0)
# [cite_start]argmin renvoie l'index de la valeur minimale [cite: 78]
meilleure_cle = np.argmin(scores_chi2)
meilleur_score = scores_chi2[meilleure_cle]

print(f"\nRÉSULTAT DE L'ANALYSE :")
print(f"La clé la plus probable est : {meilleure_cle}")
print(f"Score Chi-carré associé : {meilleur_score:.4f}")

# Déchiffrement final
message_clair = dechiffre_texte(message_chiffre, meilleure_cle)
print("\n--- Message déchiffré ---")
print(message_clair)

# --- 5. CLASSEMENT DES CLÉS (Optionnel - demandé partie 2.2) ---
[cite_start]# [cite: 104]
print("\n--- Top 5 des clés les plus probables ---")
indices_tries = np.argsort(scores_chi2) # Trie les index du plus petit score au plus grand
for i in range(5):
    cle = indices_tries[i]
    print(f"#{i+1} : Clé {cle} (Score: {scores_chi2[cle]:.2f})")
    